In [1]:
from LowLevelAnalysis import ConfigItem, ImageExtractorConfig, TailcutConfig, LowLevelAnalysis
from LowLevelData import DL0Data

import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
import math
import numpy as np
import numpy.ma as ma
import os
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from pathlib import Path
import tables

import astropy.units as u
from astropy.io import fits
from astropy.table import Table, vstack

from traitlets.config.loader import Config
from ctapipe_io_lst import LSTEventSource
from ctapipe.instrument import CameraGeometry
from ctapipe.visualization import CameraDisplay
from ctapipe.image import hillas_parameters

from lstchain.io.config import read_configuration_file
import lstchain.reco.utils as utils
from lstchain.reco import r0_to_dl1
from lstchain.io.io import dl1_images_lstcam_key, dl1_params_tel_mon_ped_key, dl1_params_tel_mon_cal_key, dl1_params_lstcam_key, dl1_params_src_dep_lstcam_key

from ctapipe.utils import get_dataset_path
from ctapipe.io import EventSource
from ctapipe.io.eventseeker import EventSeeker
#import astropy.units as u
#from copy import deepcopy

from scipy.stats import binned_statistic



In [2]:
from logging import getLogger,StreamHandler,DEBUG,INFO,WARNING,ERROR,CRITICAL

##### Logger #####
logger = getLogger(__name__)
handler = StreamHandler()
loglevel = 'INFO'
handler.setLevel(loglevel)
logger.setLevel(loglevel)
logger.addHandler(handler)

In [3]:
from lstchain import __version__ as lstchain_version
logger.info(lstchain_version)

0.7.3.post19+gitdeabb5c


In [4]:
# Matplotlib setup
plt.rcParams["font.size"] = 13

LINE_STYLES = ["solid", "dashed", "dashdot", "dotted"]
MARKER_STYLES = ['o', 's', 'x', '+', 'D', 'X', 'p', 'd', '<', '>', '^', 'v', 'H']

#plt.xkcd()

In [5]:
# Replace OUTPUT_DIR_PATH by your directory
OUTPUT_DIR_PATH = Path('/home/mitsunari.takahashi/Work/analysis/low-level/LowLevelAnalyses')
if not OUTPUT_DIR_PATH.exists():
    os.makedirs(OUTPUT_DIR_PATH)

# Config of Tailcut

In [6]:
tailcut_config = TailcutConfig({"picture_thresh":6, "boundary_thresh":3, \
                                "sigma":2.5, "keep_isolated_pixels":"false", \
                                "min_number_picture_neighbors":2, \
                                "use_only_main_island":"false", \
                                "delta_time":2})

# Config of ImageExtracter

## List of confugurations to evaluate  

In [7]:
image_extractor_configs = []

# LocalPeakWindowSum
for window_width in (5,):
    window_shift = int(window_width/2)
    image_extractor_configs.append(ImageExtractorConfig('LocalPeakWindowSum', 
                                                        {'window_shift':window_shift,
                                                         'window_width':window_width,
                                                         'apply_integration_correction':'true'}
                                                       , abbreviation='LPWS'))
    
# NeighborPeakWindowSum
for window_width in (5,):
    window_shift = int(window_width/2)
    for lwt in (0, ):#1):
        image_extractor_configs.append(ImageExtractorConfig('NeighborPeakWindowSum', 
                                                            {'window_shift':window_shift, 
                                                             'window_width':window_width,
                                                             'lwt':lwt,
                                                             'apply_integration_correction':'true'}
                                                       , abbreviation='NPWS'))
        
# SlidingWindowMaxSum
for window_width in (5,):
    image_extractor_configs.append(ImageExtractorConfig('SlidingWindowMaxSum', 
                                                        {'window_width':window_width,
                                                         'apply_integration_correction':'true'}
                                                       , abbreviation='SWMS'))

# TwoPassWindowSum
for core_threshold in (6,):
    image_extractor_configs.append(ImageExtractorConfig('TwoPassWindowSum', 
                                                        {'core_threshold':core_threshold, 
                                                         'disable_second_pass':'false',
                                                         'apply_integration_correction':'true'}
                                                       , abbreviation='TPWS'))
    
# FullWaveformSum
image_extractor_configs.append(ImageExtractorConfig('FullWaveformSum'
                                                       , abbreviation='FWS'))

In [8]:
# Write the config files down     
lowlevel_configs= []
for ie in image_extractor_configs:
    lowlevel_configs.append(LowLevelAnalysis(configs=[ie, tailcut_config], 
                                             output_dir_path=OUTPUT_DIR_PATH, 
                                             config_dir_path=OUTPUT_DIR_PATH))
for c in lowlevel_configs:    
    c.write_configfile()

{'__IMAGE_EXTRACTOR__': 'LocalPeakWindowSum', '__IMAGE_EXTRACTOR_PARAMETER_BLOCK__': '"LocalPeakWindowSum":{\n    "window_shift": 2,\n    "window_width": 5,\n    "apply_integration_correction": true\n  },'}
{'__TAILCUT__': '  "tailcut": {\n    "picture_thresh":6,\n    "boundary_thresh":3,\n    "keep_isolated_pixels":false,\n    "min_number_picture_neighbors":2,\n    "use_only_main_island":false,\n    "delta_time":2\n},\n', '__TAILCUTS_CLEAN_WITH_PEDESTAL_THRESHOLD__': '  "tailcuts_clean_with_pedestal_threshold": {\n    "picture_thresh":6,\n    "boundary_thresh":3,\n    "sigma":2.5,\n    "keep_isolated_pixels":false,\n    "min_number_picture_neighbors":2,\n    "use_only_main_island":false,\n    "delta_time":2\n},\n'}
{'__IMAGE_EXTRACTOR__': 'NeighborPeakWindowSum', '__IMAGE_EXTRACTOR_PARAMETER_BLOCK__': '"NeighborPeakWindowSum":{\n    "window_shift": 2,\n    "window_width": 5,\n    "lwt": 0,\n    "apply_integration_correction": true\n  },'}
{'__TAILCUT__': '  "tailcut": {\n    "picture_

# Shower event selection

In [9]:
EMIN = 10 *u.GeV
EMAX = 50 *u.GeV

# MC DL0 data

In [10]:
dl0_paths = [Path('/fefs/aswg/data/mc/DL0/20200629_prod5_trans_80/proton/zenith_20deg/south_pointing/proton_20deg_180deg_run1___cta-prod5-lapalma_4LSTs_MAGIC_desert-2158m_mono.simtel.gz'),
             Path('/fefs/aswg/data/mc/DL0/20200629_prod5_trans_80/proton/zenith_20deg/south_pointing/proton_20deg_180deg_run2___cta-prod5-lapalma_4LSTs_MAGIC_desert-2158m_mono.simtel.gz')]

#dl0_path = Path('/fefs/aswg/workspace/yoshiki.ohtani/Data/LaPalma/4LSTs_MAGIC/gamma-diffuse/zenith_20deg/south_pointing/run1000/sim_telarray_v3_trans_80%/cta-prod5-lapalma_4LSTs_MAGIC/0.0deg/Data/gamma_20deg_180deg_run1000___cta-prod5-lapalma_4LSTs_MAGIC_desert-2158m_mono_cone6.simtel.gz')
#dl0_path = Path('/fefs/aswg/data/mc/DL0/20200629_prod5/gamma-diffuse/zenith_20deg/south_pointing/gamma_20deg_180deg_run1___cta-prod5-lapalma_4LSTs_MAGIC_desert-2158m_mono_cone6.simtel.gz')

In [11]:
tel_id = 1 #LST-1
allowed_tels = {tel_id}

In [12]:
dl0_data = DL0Data(name='MC Proton', tel_id=tel_id)
dl0_data.add_files(file_paths=dl0_paths)

Adding [PosixPath('/fefs/aswg/data/mc/DL0/20200629_prod5_trans_80/proton/zenith_20deg/south_pointing/proton_20deg_180deg_run1___cta-prod5-lapalma_4LSTs_MAGIC_desert-2158m_mono.simtel.gz'), PosixPath('/fefs/aswg/data/mc/DL0/20200629_prod5_trans_80/proton/zenith_20deg/south_pointing/proton_20deg_180deg_run2___cta-prod5-lapalma_4LSTs_MAGIC_desert-2158m_mono.simtel.gz')]...
29 events has been added.


# DL1 data

In [13]:
for lowlevel_config in lowlevel_configs:
    logger.info(lowlevel_config.name)
    for dl0_path in dl0_data.file_paths:
        dl1_production_result = lowlevel_config.produce_mc_dl1(dl0_path=dl0_path)
        if dl1_production_result!=0:
            logger.error('Producing DL1 file failed!!')

LocalPeakWindowSum_window_shift2_window_width5_apply_integration_correctiontrue_Tailcut_picture_thresh6_boundary_thresh3_sigma2.5_keep_isolated_pixelsfalse_min_number_picture_neighbors2_use_only_main_islandfalse_delta_time2
/home/mitsunari.takahashi/Work/analysis/low-level/LowLevelAnalyses/LocalPeakWindowSum_window_shift2_window_width5_apply_integration_correctiontrue_Tailcut_picture_thresh6_boundary_thresh3_sigma2.5_keep_isolated_pixelsfalse_min_number_picture_neighbors2_use_only_main_islandfalse_delta_time2/dl1_proton_20deg_180deg_run1___cta-prod5-lapalma_4LSTs_MAGIC_desert-2158m_mono.h5 already exists! It was modified 4 hr 23 min before.

Found duplicated column obs_id, skipping
Found duplicated column event_id, skipping

NeighborPeakWindowSum_window_shift2_window_width5_lwt0_apply_integration_correctiontrue_Tailcut_picture_thresh6_boundary_thresh3_sigma2.5_keep_isolated_pixelsfalse_min_number_picture_neighbors2_use_only_main_islandfalse_delta_time2
/home/mitsunari.takahashi/Work/an

## DL1 Data Readout

In [ ]:
HILLASES_OF_INTEREST = ['intensity', 'length', 'width']

In [ ]:
for lowlevel_config in lowlevel_configs:
    lowlevel_config.read_dl1(event_used, tel_id=1, hillas_parameters=HILLASES_OF_INTEREST)#emin=EMIN, emax=EMAX)
    if len(lowlevel_config.dl1_reco_phe)!=len(true_phe):
        logger.critical('The pulse number of DL0 and DL1 does not match!!!')

In [ ]:
for lowlevel_config in lowlevel_configs:
    lowlevel_config.calc_correlations(true_phe)

# Plot

In [ ]:
lowlevel_config_benchmark = lowlevel_configs[0]

fig, axes = plt.subplots(6, 2, figsize=(10, 25))
fig.suptitle('True shower energy: {0:.1f}-{1:.1f} GeV'.format(EMIN.to(u.GeV).value, EMAX.to(u.GeV).value), fontsize=16)
for i, lowlevel_config in enumerate(lowlevel_configs):
    
    iax = 0
    
    # Linear X-axis
    xvalues = (lowlevel_config.reco_stats['mean'][1][:-1])
    
    # Log X-axis    
    logxvalues = (lowlevel_config.reco_stats_log['mean'][1][:-1])
        
    # Event count 
    axes[iax][0].errorbar(xvalues, \
                          lowlevel_config.reco_stats['count'][0], \
                          yerr=np.sqrt(lowlevel_config.reco_stats['count'][0]), \
                          fmt=MARKER_STYLES[i%len(MARKER_STYLES)], \
                          label=lowlevel_config.abbreviation)
    axes[iax][0].grid(True, which='major', axis='both')
    axes[iax][0].set_yscale('log')
    axes[iax][0].set_xlim(-1, 10)
    axes[iax][0].set_ylim(100, 1e7)
    axes[iax][0].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][0].set_ylabel('[events]')
    #axes[iax][0].legend(loc=0)


    axes[iax][1].errorbar(logxvalues, \
                          lowlevel_config.reco_stats_log['count'][0], \
                          yerr=np.sqrt(lowlevel_config.reco_stats_log['count'][0]), \
                          fmt=MARKER_STYLES[i%len(MARKER_STYLES)], \
                          label=lowlevel_config.abbreviation)
    axes[iax][1].set_xscale('log')
    axes[iax][1].set_yscale('log')
    axes[iax][1].set_xlim(1, 1000)
    #axes[iax][1].set_ylim(1, 1000)
    axes[iax][1].grid(True, which='both', axis='both')
    axes[iax][1].set_xlabel(r'$n_{true} [phe]$')
    axes[iax][1].set_ylabel('[events]')
                         
    iax += 1
        
    axes[iax][0].errorbar(xvalues, \
                          lowlevel_config.reco_stats['mean'][0], \
                          yerr=lowlevel_config.reco_stats['std'][0]/np.sqrt(lowlevel_config.reco_stats['count'][0]), \
                          fmt=MARKER_STYLES[i%len(MARKER_STYLES)], \
                          label=lowlevel_config.abbreviation)
    axes[iax][0].grid(True, which='both', axis='both')
    axes[iax][0].set_yscale('log')
    axes[iax][0].set_xlim(-1, 10)
    axes[iax][0].set_ylim(-1, 10)
    axes[iax][0].set_xlabel(r'$n_{true} [phe]$')
    axes[iax][0].set_ylabel(r'$<n_{reco}>$ [phe]')
    axes[iax][0].legend(loc=0)


    axes[iax][1].errorbar(logxvalues, \
                          lowlevel_config.reco_stats_log['mean'][0], \
                          yerr=lowlevel_config.reco_stats_log['std'][0]/np.sqrt(lowlevel_config.reco_stats_log['count'][0]), \
                          fmt=MARKER_STYLES[i%len(MARKER_STYLES)], \
                          label=lowlevel_config.abbreviation)
    axes[iax][1].set_xscale('log')
    axes[iax][1].set_yscale('log')
    axes[iax][1].set_xlim(1, 1000)
    axes[iax][1].set_ylim(1, 1000)
    axes[iax][1].grid(True, which='both', axis='both')
    axes[iax][1].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][1].set_ylabel(r'$<n_{reco}>$ [phe]')
                         
    iax += 1

    axes[iax][0].errorbar(xvalues[1:], \
                          lowlevel_config.reco_frac_stats['mean'][0][1:], \
                          yerr=lowlevel_config.reco_frac_stats['std'][0][1:]/np.sqrt(lowlevel_config.reco_frac_stats['count'][0][1:]), \
                          fmt=MARKER_STYLES[i%len(MARKER_STYLES)], \
                    label=lowlevel_config.abbreviation)
    axes[iax][0].grid(True, which='both', axis='both')
    axes[iax][0].set_xlim(-1, 10)
    axes[iax][0].set_ylim(0, 3)
    axes[iax][0].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][0].set_ylabel(r'$<n_{reco}/n_{true}>$')

    # Log X-axis    
    axes[iax][1].errorbar(logxvalues, \
                          lowlevel_config.reco_frac_stats_log['mean'][0], \
                          yerr=lowlevel_config.reco_frac_stats_log['std'][0]/np.sqrt(lowlevel_config.reco_frac_stats_log['count'][0]), \
                          fmt=MARKER_STYLES[i%len(MARKER_STYLES)], \
                          label=lowlevel_config.abbreviation)
    axes[iax][1].set_xscale('log')
    axes[iax][1].set_xlim(1, 1000)
    axes[iax][1].set_ylim(0.8, 1.2)
    axes[iax][1].grid(True, which='both', axis='both')
    axes[iax][1].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][1].set_ylabel(r'$<n_{reco}/n_{true}>$')
                         
    iax += 1
    
    axes[iax][0].plot(xvalues, \
                    lowlevel_config.reco_stats['std'][0], MARKER_STYLES[i%len(MARKER_STYLES)], \
                    label=lowlevel_config.abbreviation)
    axes[iax][0].grid(True, which='both', axis='both')
    axes[iax][0].set_xlim(0, 10)
    axes[iax][0].set_ylim(1, 4)
    axes[iax][0].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][0].set_ylabel(r'$\sqrt{Var(n_{reco})}$ [phe]')

    axes[iax][1].plot(logxvalues, \
                    lowlevel_config.reco_stats_log['std'][0], MARKER_STYLES[i%len(MARKER_STYLES)], \
                    label=lowlevel_config.abbreviation)
    axes[iax][1].set_xscale('log')
    axes[iax][1].set_yscale('log')
    axes[iax][1].grid(True, which='both', axis='both')
    axes[iax][1].set_xlim(1, 1000)
    axes[iax][1].set_ylim(1, 100)
    axes[iax][1].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][1].set_ylabel(r'$\sqrt{Var(n_{reco})}$ [phe]')   
                         
    iax += 1
    axes[iax][0].plot(xvalues[1:], \
                    lowlevel_config.reco_stats['std'][0][1:] / \
                    (lowlevel_config.reco_stats['mean'][0][1:]-lowlevel_config.mean_phe_true0), \
                    MARKER_STYLES[i%len(MARKER_STYLES)], \
                    label=lowlevel_config.abbreviation)
    axes[iax][0].grid(True, which='both', axis='both')
    axes[iax][0].set_xlim(0, 10)
    axes[iax][0].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][0].set_ylabel('Resolution') #(r'$\sqrt{Var(n_{reco})} / (\bar{n}_{reco}-\bar{n}_{reco}(n_{true}=0))$')

    axes[iax][1].plot(logxvalues[1:], \
                    lowlevel_config.reco_stats['std'][0][1:len(logxvalues)] / \
                    (lowlevel_config.reco_stats['mean'][0][1:len(logxvalues)]-lowlevel_config.mean_phe_true0), \
                    MARKER_STYLES[i%len(MARKER_STYLES)], \
                    label=lowlevel_config.abbreviation)
    axes[iax][1].set_xscale('log')
    axes[iax][1].set_yscale('log')
    axes[iax][1].grid(True, which='both', axis='both')
    axes[iax][1].set_xlim(1, 1000)
    #axes[iax][0].set_ylim(0, 3)
    axes[iax][1].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][1].set_ylabel('Resolution')#r'$\sqrt{Var(n_{reco})} / (\bar{n}_{reco}-\bar{n}_{reco}(n_{true}=0))$')
    
    iax += 1
    axes[iax][0].plot(xvalues[1:], \
                      lowlevel_config.reco_stats['std'][0][1:] / \
                      lowlevel_config_benchmark.reco_stats['std'][0][1:] / \
                      (lowlevel_config.reco_stats['mean'][0][1:]-lowlevel_config.mean_phe_true0) * \
                      (lowlevel_config_benchmark.reco_stats['mean'][0][1:]-lowlevel_config_benchmark.mean_phe_true0), \
                      MARKER_STYLES[i%len(MARKER_STYLES)], label=lowlevel_config.abbreviation)
    axes[iax][0].grid(True, which='both', axis='both')
    axes[iax][0].set_xlim(0, 10)
    axes[iax][0].set_ylim(0.25, 1.75)
    axes[iax][0].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][0].set_ylabel('Resolution relative to {0}'.format(lowlevel_config_benchmark.abbreviation))    

    axes[iax][1].plot(logxvalues[1:], \
                      lowlevel_config.reco_stats['std'][0][1:len(logxvalues)] / \
                      lowlevel_config_benchmark.reco_stats['std'][0][1:len(logxvalues)] / \
                      (lowlevel_config.reco_stats['mean'][0][1:len(logxvalues)]-lowlevel_config.mean_phe_true0) * \
                      (lowlevel_config_benchmark.reco_stats['mean'][0][1:len(logxvalues)]-lowlevel_config_benchmark.mean_phe_true0), \
                      MARKER_STYLES[i%len(MARKER_STYLES)], label=lowlevel_config.abbreviation)
    axes[iax][1].set_xscale('log')
    #axes[iax][1].set_yscale('log')
    axes[iax][1].grid(True, which='both', axis='both')
    axes[iax][1].set_xlim(1, 1000)
    #axes[iax][0].set_ylim(0, 3)
    axes[iax][1].set_xlabel(r'$n_{true}$ [phe]')
    axes[iax][1].set_ylabel('Resolution relative to {0}'.format(lowlevel_config_benchmark.abbreviation))
    
axes[0][0].legend(loc=0)   
axes[2][0].legend(loc=0)
axes[5][1].legend(loc=0)
plt.tight_layout()    

## Distribution for specific-phe-number events

In [ ]:
list_true_phes = [0, 3, 6, 10] #List of the true photoelectron numbers to draw the distribution of the reconstructed phe
mask_truephe = {}
true_phe_bins = np.linspace(-10, 30, 81)

for tphe in list_true_phes:
    mask_truephe[tphe] = np.array(true_phe!=tphe)
    logger.debug('True {0}-phe events: {1}'.format(tphe, sum(1-mask_truephe[tphe])))

for i, lowlevel_config in enumerate(lowlevel_configs):
    
    for tphe in list_true_phes:
        lowlevel_config.reco_phe_hists[tphe] = np.histogram(lowlevel_config.dl1_reco_phe, \
                                                            weights=[float(1-m) for m in mask_truephe[tphe]], \
                                                            bins=true_phe_bins)

In [ ]:
fig, axes = plt.subplots(len(lowlevel_configs), 1, figsize=(8, 4*len(lowlevel_configs)))
for i, image_extracter_config in enumerate(lowlevel_configs):
    for j, tphe in enumerate(list_true_phes):
        axes[i].hist(lowlevel_config.reco_phe_hists[tphe][1][:-1], \
                        lowlevel_config.reco_phe_hists[tphe][1], \
                        weights=lowlevel_config.reco_phe_hists[tphe][0]/sum(lowlevel_config.reco_phe_hists[tphe][0]), \
                        label=r'$n_{{true}}={{{0}}}$ phe'.format(tphe), histtype='step')#, alpha=0.2)
    axes[i].set_title(lowlevel_config.abbreviation)
    axes[i].grid(True, which='major', axis='x')
    axes[i].set_xlabel(r'$n_{reco}$ [phe]')
    axes[i].legend(loc=0)    
plt.tight_layout()  

## ROC Curve

In [ ]:
sig_phes = [3, 6, 10]
fig, axes = plt.subplots(len(sig_phes), 1, figsize=(10, 6*len(sig_phes)))

for i, sig_phe in enumerate(sig_phes):
    for j, lowlevel_config in enumerate(lowlevel_configs):
        xvals, yvals = lowlevel_config.get_roc_curve(sig_phe=sig_phe)
        axes[i].plot(xvals, yvals, label=lowlevel_config.abbreviation, ls=LINE_STYLES[j%len(LINE_STYLES)])
    axes[i].set_title(r'$n_{{true}}={0}$ phe'.format(sig_phe))
    axes[i].grid(True, which='major', axis='both')
    axes[i].set_yscale('log')
    axes[i].set_ylim(1e-3, 1)
    axes[i].set_xlabel('Signal acceptance')
    axes[i].set_ylabel('Background residual')
    axes[i].legend(loc=0)    
plt.tight_layout()  

# Separation power evaluation
Not working well yet

In [ ]:
for i, lowlevel_config in enumerate(lowlevel_configs):
    logger.info(lowlevel_config.name)
    lowlevel_config.find_best_separation('error', [0,3])

In [ ]:
for i, lowlevel_config in enumerate(lowlevel_configs):
    logger.info(lowlevel_config.name)
    lowlevel_config.find_best_separation('error', [0,6])

In [ ]:
for i, lowlevel_config in enumerate(lowlevel_configs):
    logger.info(lowlevel_config.name)
    lowlevel_config.find_best_separation('error', [0,10])

# Hillas Parameters

In [ ]:
fig, axes = plt.subplots(len(HILLASES_OF_INTEREST), 1, figsize=(10, 6*len(HILLASES_OF_INTEREST)))

for i, param in enumerate(HILLASES_OF_INTEREST):
    for j, lowlevel_config in enumerate(lowlevel_configs):
        vals = lowlevel_config.parameter_value_dict[param]
        if j==0:
            hist_result = axes[i].hist(vals, lw=2, bins=20, histtype="step", label=lowlevel_config.abbreviation)
        else:
            axes[i].hist(vals, lw=2, bins=hist_result[1], histtype="step", label=lowlevel_config.abbreviation)
    #axes[i].set_title(param)
    axes[i].grid(True, which='major', axis='both')
    #axes[i].set_yscale('log')
    #axes[i].set_ylim(1e-3, 1)
    axes[i].set_xlabel(param)
    axes[i].set_ylabel('[events]')
    axes[i].legend(loc=0)    
plt.tight_layout()  